In [1]:
''' To run on GPU Select 1, to Run on CPU select -1'''

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import random
import numpy as np 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [4]:
'''from numba import cuda

device = cuda.get_current_device()
attribs = [s for s in dir(device) if s.isupper()]

for attr in attribs:
    print(attr, '=', getattr(device, attr))
'''

'''from numba import cuda

device = cuda.get_current_device()
gpu_name = device.name.decode('utf-8')
print("GPU Name:", gpu_name)'''

'''device = cuda.get_current_device()
attribs = [s for s in dir(device) if s.isupper()]

for attr in attribs:
    print(attr, '=', getattr(device, attr))
'''

"from numba import cuda\n\ndevice = cuda.get_current_device()\nattribs = [s for s in dir(device) if s.isupper()]\n\nfor attr in attribs:\n    print(attr, '=', getattr(device, attr))\n"

In [7]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath, 'rb').read().decode(encoding = 'utf-8').lower()

In [8]:
text = text[150000:750000]
characters = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [9]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [10]:
sentences = []
next_characters = []

In [15]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

In [16]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)),dtype= np.bool_ )
y = np.zeros((len(sentences), len(characters)), dtype= np.bool_ )

In [17]:
for i,sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1

In [18]:
model = Sequential()
model.add(LSTM(128, input_shape = (SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer = RMSprop(lr=0.02))

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
with tf.device('/GPU:1'):
    model.fit(x, y, batch_size = 256, epochs = 4)

In [ ]:
'''with tf.device('/CPU'):
    model.fit(x, y, batch_size = 256, epochs = 20)
    '''

Epoch 1/20
782/782 [==============================] - 51s 63ms/step - loss: 1.9984
Epoch 2/20
782/782 [==============================] - 48s 62ms/step - loss: 1.6025
Epoch 3/20
782/782 [==============================] - 48s 61ms/step - loss: 1.5232
Epoch 4/20
782/782 [==============================] - 48s 61ms/step - loss: 1.4794
Epoch 5/20
782/782 [==============================] - 48s 61ms/step - loss: 1.4525
Epoch 6/20
782/782 [==============================] - 47s 60ms/step - loss: 1.4318
Epoch 7/20
782/782 [==============================] - 48s 61ms/step - loss: 1.4145
Epoch 8/20
782/782 [==============================] - 47s 61ms/step - loss: 1.4020
Epoch 9/20
782/782 [==============================] - 48s 62ms/step - loss: 1.3905
Epoch 10/20
782/782 [==============================] - 49s 62ms/step - loss: 1.3805
Epoch 11/20
782/782 [==============================] - 49s 63ms/step - loss: 1.3741
Epoch 12/20
782/782 [==============================] - 49s 63ms/step - loss: 1.3654
E

In [ ]:
model.save('shakespeares_gen.model')
model.save_weights('shakespeares_gen.weights')

INFO:tensorflow:Assets written to: shakespeares_gen.model\assets


INFO:tensorflow:Assets written to: shakespeares_gen.model\assets


In [20]:
model = tf.keras.models.load_model('shakespeares_gen.model')

In [25]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [26]:
def generate_text( length, temperature):
    start_index = random.randint( 0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model.predict(x, verbose = 0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [27]:
with tf.device('/CPU'):
    print(generate_text(300,0.2))

C:\Users\user\AppData\Local\Temp\ipykernel_1936\4129162411.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


er:
my lord, the mayor of london comes to thee.

king richard iii:
and so stay the grown of the bolingbroke.

king richard iii:
the send the compless of the world and the servance.

king richard iii:
the send the send the send to the complious son
is provent of the servant of the mark to thee.

king richard iii:
and thou should the man of


In [28]:
with tf.device('/CPU'):
    print(generate_text(300,0.6))

C:\Users\user\AppData\Local\Temp\ipykernel_1936\4129162411.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ickness that i have;
and thy unkindness with here as a wound
and what see the statter and most thou must but of my lord.

nor many and as my husband,
and have servantge of the blass of the ground
should no state the way but a i repose sen.

king richard iii:
and ludy the must to the houses, and the rest;
and should the but no pity the cam
